# Notebook 기본 세팅

In [21]:
# Constant 선언

# 프로젝트 루트 디렉토리를 식별하기 위한 마커 파일 이름
ROOT_MARKER = "pyproject.toml"

# 한글 표시를 위한 나눔바른고딕 폰트 파일 이름
# matplotlib 의 font_manager 에 실제 폰트 파일의 위치를 넣어주어야 한다.
KOREAN_FONT_FILE = "NanumBarunGothic.ttf"

# matplotlib 에서는 font-family 의 이름으로 font 를 설정한다.
# 그래서 font 파일 그 자체가 아니라, 그 파일의 family 이름을 적어준다.
KOREAN_FONT_FAMILY = "NanumBarunGothic"

# 참고
# Font Family 와 Font File 의 차이는,
# Font Family 는 비슷한 디자인 특성을 공유하는 글꼴 그룹을 의미한다.
#
# 예를 들어 '나눔바른고딕' 폰트 패밀리는 일반(Regular), 굵게(Bold), 기울임(Italic) 등 여러 스타일을 포함할 수 있다.
# 반면, 폰트 파일(.ttf, .otf 등)은 이러한 폰트의 하나의 스타일이 저장된 실제 파일이다.
#
# 이 프로젝트에서는 폰트 용량을 줄이기 위해 일반(Regular) 인 NanumBarunGothic.ttf 만 사용한다.

In [22]:
# 프로젝트 root 를 sys.path 에 추가해서 import 구문을 사용하기 쉽게
from pathlib import Path


def find_project_root() -> Path:
    """
    pyproject.toml 파일을 기준으로 루트 디렉토리를 찾는다.
    :return: Path: 프로젝트 루트 디렉토리 경로
    """

    current_path = Path().resolve()

    while current_path != current_path.parent:
        if (current_path / ROOT_MARKER).exists():
            return current_path

        current_path = current_path.parent

    raise FileNotFoundError("프로젝트 루트 디렉토리를 찾을 수 없습니다.")


ROOT_DIR = find_project_root()

In [23]:
# matplotlib 의 한글 font 설정
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt


FONTS_DATA_DIR = ROOT_DIR / "notebooks" / "fonts"


def setup_korean_font():
    font_path = FONTS_DATA_DIR / KOREAN_FONT_FILE
    fm.fontManager.addfont(font_path)

    # 폰트 설정
    plt.rcParams["font.family"] = KOREAN_FONT_FAMILY
    plt.rcParams["axes.unicode_minus"] = False


setup_korean_font()

# EDA & 전처리 
1. NCloud에서 원본 데이터 가져오기
2. 사용할 Feature만 선택
3. 결측치 및 이상치 처리
4. 파생변수 생성
5. 범주형/연속형 분리 및 인코딩

In [24]:
import os

import boto3
import pandas as pd
from botocore.client import Config


# 환경변수 불러오기
NCLOUD_ACCESS_KEY = os.getenv("NCLOUD_ACCESS_KEY")
NCLOUD_SECRET_KEY = os.getenv("NCLOUD_SECRET_KEY")
NCLOUD_STORAGE_REGION = os.getenv("NCLOUD_STORAGE_REGION")
NCLOUD_STORAGE_BUCKET = os.getenv("NCLOUD_STORAGE_BUCKET")
NCLOUD_STORAGE_ENDPOINT_URL = os.getenv("NCLOUD_STORAGE_ENDPOINT_URL")

# S3 클라이언트 생성
s3_client = boto3.client(
    "s3",
    endpoint_url=NCLOUD_STORAGE_ENDPOINT_URL,
    aws_access_key_id=NCLOUD_ACCESS_KEY,
    aws_secret_access_key=NCLOUD_SECRET_KEY,
    region_name=NCLOUD_STORAGE_REGION,
    config=Config(signature_version="s3v4"),
)

# 데이터 저장 경로
DATASETS_DIR = "datasets"

In [25]:
# # 1. NCloud에서 저장된 CSV 불러오기
# def read_csv_as_dataframe(key: str) -> pd.DataFrame:
#     r = s3_client.get_object(Bucket=NCLOUD_STORAGE_BUCKET, Key=key)
#     file_content = r["Body"].read().decode("utf-8")
#     csv_buffer = StringIO(file_content)
#     return pd.read_csv(csv_buffer)

# df = read_csv_as_dataframe(f"{DATASETS_DIR}/weather_all_station.csv")

In [26]:
# CSV 불러오기
df = pd.read_csv("weather_fulldata.csv")

/tmp/ipykernel_4883/3767135182.py:2: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("weather_fulldata.csv")


In [27]:
selected_features = [
    "tm",  # 날짜 정보
    "stnNm",  # 지점 정보
    "avgTa",
    "minTa",
    "maxTa",  # 기온 관련
    "avgWs",
    "maxWs",  # 풍속
    "avgRhm",
    "avgTd",
    "avgTca",  # 습도 관련
    "avgPv",
    "avgPa",
    "avgPs",  # 증기압/기압
    "iscs",  # 예측 대상
]
df_filtered = df[selected_features]

# stnNm, iscs 제외하고 나머지 컬럼을 수치형으로 변환
for col in df_filtered.columns:
    if col not in ["stnNm", "iscs", "tm"]:
        df_filtered[col] = pd.to_numeric(df_filtered[col], errors="coerce")

/tmp/ipykernel_4883/3059279621.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[col] = pd.to_numeric(df_filtered[col], errors="coerce")


In [28]:
# 3. 결측치 및 이상치 처리
# 변수 분류
categorical_cols = ["stnNm", "iscs", "tm"]
continuous_cols = [col for col in df_filtered.columns if col not in categorical_cols]

# 결측치 처리
# 범주형 → 'NULL'로 채움
df_filtered[categorical_cols] = df_filtered[categorical_cols].fillna("NULL")

# 연속형 → 0으로 채움
df_filtered[continuous_cols] = df_filtered[continuous_cols].fillna(0)

# 이상치 제거 (IQR)
for col in continuous_cols:
    Q1 = df_filtered[col].quantile(0.25)
    Q3 = df_filtered[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # 이상치 제거
    before = df_filtered.shape[0]
    df_filtered = df_filtered[(df_filtered[col] >= lower_bound) & (df_filtered[col] <= upper_bound)]
    after = df_filtered.shape[0]
    print(f"{col}: {before - after}개의 이상치 제거됨")

print("\n 결측치 처리 및 이상치 제거 완료")

/tmp/ipykernel_4883/1137341733.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[categorical_cols] = df_filtered[categorical_cols].fillna("NULL")
/tmp/ipykernel_4883/1137341733.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered[continuous_cols] = df_filtered[continuous_cols].fillna(0)


avgTa: 2개의 이상치 제거됨
minTa: 0개의 이상치 제거됨
maxTa: 3개의 이상치 제거됨
avgWs: 8093개의 이상치 제거됨
maxWs: 3232개의 이상치 제거됨
avgRhm: 882개의 이상치 제거됨
avgTd: 0개의 이상치 제거됨
avgTca: 0개의 이상치 제거됨
avgPv: 0개의 이상치 제거됨
avgPa: 5913개의 이상치 제거됨
avgPs: 38개의 이상치 제거됨

 결측치 처리 및 이상치 제거 완료


In [29]:
# 4. 파생변수 생성

# 1. 일교차 (최고기온 - 최저기온)
df_filtered["temp_range"] = df_filtered["maxTa"] - df_filtered["minTa"]

# 2. 평균기온 대비 최고/최저 평균과의 편차
df_filtered["temp_dev_from_avg"] = df_filtered["avgTa"] - (df_filtered["maxTa"] + df_filtered["minTa"]) / 2

# 3. 기압차이 (평균기압 - 평균수증기압)
df_filtered["pressure_gap"] = df_filtered["avgPs"] - df_filtered["avgPv"]

# 4. 습도 대비 온도 비율 (평균기온 / 평균습도) — 0으로 나누는 것 방지
df_filtered["humidity_temperature_ratio"] = df_filtered["avgTa"] / (df_filtered["avgRhm"] + 1e-6)

# 5. 풍속 범위 (최대풍속 - 평균풍속)
df_filtered["wind_range"] = df_filtered["maxWs"] - df_filtered["avgWs"]

In [30]:
import re


def simplify_iscs(raw_text):
    """
    {비}0140-{비}{강도0}0300-0320 → 비
    여러 이벤트가 있을 경우 → 비+안개 형태로 변환
    """
    if pd.isna(raw_text) or raw_text.strip() == "":
        return "기타"

    # 중괄호 안의 한글 단어 추출 → 리스트 반환
    matches = re.findall(r"\{([가-힣]+)\}", raw_text)
    if matches:
        return "+".join(sorted(set(matches)))  # 중복 제거 및 정렬
    return "기타"

In [31]:
df_filtered = df_filtered.dropna(subset=["iscs"])
df_filtered["iscs"] = df_filtered["iscs"].apply(simplify_iscs)

In [32]:
# 5. 범주형/연속형 인코딩
import pandas as pd
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer


def preprocess_weather_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    주어진 날씨 데이터프레임에서 selected_features 기준으로 인코딩 및 전처리를 수행
    """
    selected_features = [
        "tm",
        "stnNm",
        "avgTa",
        "avgWs",
        "avgRhm",
        "avgTd",
        "avgPv",
        "avgPa",
        "avgPs",
        "temp_range",
        "temp_dev_from_avg",
        "pressure_gap",
        "humidity_temperature_ratio",
        "wind_range",
        "iscs",
    ]

    # 1. 필요한 컬럼만 선택
    df = df[selected_features].copy()

    # 2. 범주형 변수 처리
    # stnNm: Label Encoding
    le = LabelEncoder()
    df["stnNm_encoded"] = le.fit_transform(df["stnNm"])

    # iscs: MultiLabelBinarizer로 다중 이진 인코딩
    df["iscs_split"] = df["iscs"].str.split("+")
    mlb = MultiLabelBinarizer()
    iscs_encoded = pd.DataFrame(mlb.fit_transform(df["iscs_split"]), columns=[f"iscs_{cls}" for cls in mlb.classes_])

    # 3. 병합 및 불필요 컬럼 제거
    df_processed = pd.concat([df.drop(columns=["stnNm", "iscs", "iscs_split"]), iscs_encoded], axis=1)

    return df_processed

In [33]:
df_prepared = preprocess_weather_data(df_filtered)
print(df_prepared.columns)

Index(['tm', 'avgTa', 'avgWs', 'avgRhm', 'avgTd', 'avgPv', 'avgPa', 'avgPs',
       'temp_range', 'temp_dev_from_avg', 'pressure_gap',
       'humidity_temperature_ratio', 'wind_range', 'stnNm_encoded', 'iscs_가루눈',
       'iscs_기타', 'iscs_낮은안개', 'iscs_눈', 'iscs_달무리', 'iscs_달코로나', 'iscs_동우',
       'iscs_무지개', 'iscs_박무', 'iscs_비', 'iscs_소나기', 'iscs_소낙눈', 'iscs_소낙성진눈깨비',
       'iscs_싸락눈', 'iscs_싸락우박', 'iscs_안개', 'iscs_안개비', 'iscs_연기', 'iscs_연무',
       'iscs_용오름', 'iscs_우박', 'iscs_이동', 'iscs_진눈깨비', 'iscs_착빙성비', 'iscs_채운',
       'iscs_해코로나', 'iscs_햇무리', 'iscs_황사', 'iscs_회오리바람'],
      dtype='object')


In [35]:
# 전처리 완료된 데이터프레임 저장
df_prepared.to_csv("df_prepared.csv", index=False, encoding="utf-8-sig")
print("✅ 로컬에 'df_prepared.csv' 저장 완료")

✅ 로컬에 'df_prepared.csv' 저장 완료


# 결측치(2020.01.01-2025.05.27)

In [1]:
import pandas as pd


# CSV 불러오기
df = pd.read_csv("weather_fulldata.csv")

# 1. 각 열의 결측치 개수 확인
missing_counts = df.isnull().sum().sort_values(ascending=False)
print("📌 결측치 개수:")
print(missing_counts)

/tmp/ipykernel_28283/3791073739.py:4: DtypeWarning: Columns (60) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("weather_fulldata.csv")


📌 결측치 개수:
ddMefsHrmt     170735
sumDpthFhsc    170646
ddMefs         170643
ddMesHrmt      170049
ddMes          170027
                ...  
maxTaHrmt          91
ssDur              30
stnNm               0
tm                  0
stnId               0
Length: 62, dtype: int64


In [4]:
# 📌 결측치 개수 및 결측률 계산
missing_df = pd.DataFrame(
    {"missing_count": df.isnull().sum(), "missing_ratio (%)": round(df.isnull().sum() / len(df) * 100, 2)}
)

# 결측치가 있는 컬럼만 필터링하여 정렬
missing_df = missing_df[missing_df["missing_count"] > 0].sort_values(by="missing_count", ascending=False)

# 출력
print("📌 컬럼별 결측치 개수 및 비율:")
display(missing_df)

📌 컬럼별 결측치 개수 및 비율:


,missing_count,missing_ratio (%)
ddMefsHrmt,170735,99.42
sumDpthFhsc,170646,99.37
ddMefs,170643,99.37
ddMesHrmt,170049,99.02
ddMes,170027,99.01
sumFogDur,168967,98.39
n99Rn,156962,91.40
sumRnDur,155226,90.39
avgM30Te,147696,86.01
avgM50Te,147691,86.00


| 구분       | 변수                            | 설명                     | 결측률      | 사용 여부                             |
|------------|----------------------------------|--------------------------|-------------|----------------------------------------|
| 지점       | `stnNm`                         | 지역명                  | 0%          | ✅                                     |
| 기온       | `avgTa`, `minTa`, `maxTa`       | 평균/최고/최저 기온     | ~0.05%      | ✅                                     |
| 강수       | `sumRn`, `sumRnDur`, `hr1MaxRn` | 강수량, 지속시간        | 60~90%      | ⚠️ (보간 또는 예외 처리 고려)        |
| 적설       | `ddMes`, `ddMefs`, `sumDpthFhsc`| 적설량                  | 99% 이상     | ❌                                     |
| 풍속       | `avgWs`, `maxWs`                | 평균/최대 풍속          | <1%         | ✅                                     |
| 습도       | `avgRhm`, `avgTd`, `avgTca`     | 상대습도, 이슬점, 운량  | ~1%         | ✅                                     |
| 증기압/기압| `avgPv`, `avgPa`, `avgPs`       | 수증기량 관련            | <0.2%       | ✅                                     |
| 안개       | `sumFogDur`                     | 안개 지속시간           | 98% 이상     | ❌                                     |
| 일사량     | `sumGsr`, `hr1MaxIcsr`, `ssDur` | 햇빛 관련               | ~50% 이하   | ⚠️ (조건부 포함 가능)                |
| 타겟       | `iscs`                          | 예측 대상               | 85%         | ✅ (학습용 결측 제거 필요)            |

In [ ]:
selected_features = [
    # 지점 정보
    "stnNm",
    # 기온 관련
    "avgTa",
    "minTa",
    "maxTa",
    # 풍속
    "avgWs",
    "maxWs",
    # 습도 관련
    "avgRhm",
    "avgTd",
    "avgTca",
    # 증기압/기압
    "avgPv",
    "avgPa",
    "avgPs",
    # 예측 대상
    "iscs",
]

In [ ]:
optional_features = [
    "sumRn",
    "sumRnDur",
    "hr1MaxRn",  # 강수량 정보
    "sumGsr",
    "hr1MaxIcsr",
    "ssDur",  # 일사량
]